# Data Loader

Script to read in CSV files and export to PostgreSQL.

In [1]:
#Not all these dependencies are needed to load the data, but these are good for notebook analysis

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sqlalchemy import create_engine

pd.set_option('max_columns', 500)

In [2]:
def importer(path):
    '''Reads in and formats CSV accordingly'''
    df = pd.read_csv(path)
    
    return df

path = './StateEthicsReport.csv'

campaign = importer(path)

In [3]:
campaign.head()

,FilerID,Type,LastName,FirstName,Address,City,State,Zip,PAC,Occupation,Employer,Date,Election,Election_Year,Cash_Amount,In_Kind_Amount,In_Kind_Description,Candidate_FirstName,Candidate_MiddleName,Candidate_LastName,Candidate_Suffix,Committee_Name
0,C2017000285,Monetary,Arford-Tasker,Nancy,9175 SW Arrow Wood Dr,Portland,OR,97223-7265,NaN,Retired,Retired,8/30/2018 12:00:00 AM,General,2018,10.0,0.0,NaN,Stacey,Yvonne,Abrams,NaN,Stacy Abrams for Governor
1,C2017000285,Monetary,Arias,L. Patricia,2112 Chesterfield Dr NE,Atlanta,GA,30345-3706,NaN,Attorney,"Orange Business Services US, Inc.",8/10/2018 12:00:00 AM,General,2018,15.0,0.0,NaN,Stacey,Yvonne,Abrams,NaN,Stacy Abrams for Governor
2,C2017000285,Monetary,Anderson,Ann S,2002 Amberley Dr,Evans,GA,30809-6752,NaN,Not Employed,Not Employed,7/14/2018 12:00:00 AM,General,2018,125.0,0.0,NaN,Stacey,Yvonne,Abrams,NaN,Stacy Abrams for Governor
3,C2017000285,Monetary,Anderson,Carol,2418 Boulder Rd SE,Atlanta,GA,30316-3673,NaN,Professor,Emory University,7/20/2018 12:00:00 AM,General,2018,50.0,0.0,NaN,Stacey,Yvonne,Abrams,NaN,Stacy Abrams for Governor
4,C2017000285,Monetary,Anderson,Joseph,2646 Nakai Trl,Flagstaff,AZ,86005-3660,NaN,College Professor,Northern Ariz university,9/26/2018 12:00:00 AM,General,2018,250.0,0.0,NaN,Stacey,Yvonne,Abrams,NaN,Stacy Abrams for Governor


In [4]:
def id_maker(df, col_name, identity_fields, base_number=1000000000):
    '''Read in DataFrame and a list of fields used to identify an entity. Assign a number to those fields.'''
    
    match_table = df[identity_fields].drop_duplicates().copy()
    match_table[col_name] = np.arange(base_number, base_number+len(match_table))
    
    df = pd.merge(df,
                  match_table,
                  on=identity_fields,
                  how='left')
    
    return df[col_name]



#id_check = id_maker(df = campaign, col_name = 'ContributorId', identity_fields = ['LastName', 'FirstName', 'Zip', 'Address'])

In [5]:
def contributor(df, key_col='ContributorId'):
    '''Parse and prep contributor info from the database. Contributor schema:
    
    ContributorId
    LastName
    FirstName
    Address1
    Address2
    City
    State
    Zip
    PAC
    Occupation
    Employer'''
    
    contributor = df.copy() #copy dataframe so as not to impact the data in the original location
    
    #Prepare dataframe to use same fields as in schema
    contributor_fields = ['ContributorId', 'LastName', 'FirstName', 'Address1', 'Address2',
                          'City', 'State', 'Zip', 'PAC', 'Occupation', 'Employer']
    
    contributor[key_col] = id_maker(df=contributor, 
                                    col_name=key_col, 
                                    identity_fields=['LastName', 'FirstName', 'Address', 'Zip'])
    
    contributor['Address1'] = contributor['Address']
    contributor['Address2'] = np.nan
    
    contributor = contributor[contributor_fields].drop_duplicates(subset=['ContributorId']).reset_index(drop=True)
    
    return contributor


In [6]:
contributors = contributor(campaign)

In [7]:
#Builds the key to connect with Postgres Database using sqlalchemy and pandas. password can be string of choice, make sure you preserve the punctuation in each string
file = open('getin.txt')
driver = 'postgresql+pg8000://'
user = 'postgres:'
pwd = file.read()
host = '@db'
database = '/electionmoney'

key = driver + user + pwd + host + database
key

'postgresql+pg8000://postgres:pw@db/electionmoney'

In [8]:
#Establish the connection using the key

from sqlalchemy import create_engine
engine = create_engine(key)
connection = engine.connect()

In [9]:
connection

In [10]:
contributors.to_sql('Contributor', connection, if_exists='append', index=False) #send the table, appending rows that already exist rather than deleting and replacing

In [11]:
pd.read_sql('Contributor', connection) #readout of the data from the SQL database

,ContributorId,LastName,FirstName,Address1,Address2,City,State,Zip,PAC,Occupation,Employer
0,1000000000,Arford-Tasker,Nancy,9175 SW Arrow Wood Dr,NaN,Portland,OR,97223-7265,NaN,Retired,Retired
1,1000000001,Arias,L. Patricia,2112 Chesterfield Dr NE,NaN,Atlanta,GA,30345-3706,NaN,Attorney,"Orange Business Services US, Inc."
2,1000000002,Anderson,Ann S,2002 Amberley Dr,NaN,Evans,GA,30809-6752,NaN,Not Employed,Not Employed
3,1000000003,Anderson,Carol,2418 Boulder Rd SE,NaN,Atlanta,GA,30316-3673,NaN,Professor,Emory University
4,1000000004,Anderson,Joseph,2646 Nakai Trl,NaN,Flagstaff,AZ,86005-3660,NaN,College Professor,Northern Ariz university
5,1000000005,Anderson,Lee,50 Yacht Cove Dr,NaN,Hilton Head Island,SC,29928-1518,NaN,Retired,Retired
6,1000000006,Appleby,Elizabeth P.,68 Maple Ave NW,NaN,Marietta,GA,30064-2220,NaN,Retired,None
7,1000000007,Applegate,Mary,1207 Ridge Ave N,NaN,Tifton,GA,31794-3955,NaN,Not Employed,Not Employed
8,1000000008,Arce,Ellen,PO box 588,NaN,Marshfield,MA,02050-0588,NaN,pharmacist,Cognosante LLC
9,1000000009,Archer,Kimberly,863 Forest Ridge Dr SE,NaN,Marietta,GA,30067-7176,NaN,Systems analyst,Secureworks inc.
